In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa as tsa
import statsmodels as sm
from datetime import datetime
import os
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse


In [ ]:
label_regular = 'Gasolina regular'

In [ ]:
dfToUse = "consumo"

def returnQuantRows(dfToUse):
    if (dfToUse == 'consumo'):
        return 269 
    return 257

toUse = returnQuantRows(dfToUse)

In [ ]:
df = pd.read_excel(dfToUse+'.xlsx', engine='openpyxl')
df = df[['Fecha', label_regular]]

In [ ]:
df = df[:toUse]
df['Fecha'] = pd.to_datetime(df['Fecha'])

In [ ]:
quant_vars = [label_regular]
df[quant_vars].astype(float).describe()

# Modelo ARIMA -de la hoja pasada-

### Separando test y train

In [ ]:
rows = len(df)
train_df = df[0:rows-17] # 2000-01-01 a 2020-12-01
test_df = df[rows-17:] # 2021-01-01	 a 2022-05-01	
print(len(train_df), len(test_df))

In [ ]:
def make_timeline(column):
  plt.rcParams["figure.figsize"] = (20,5.5)
  mediaGasoline = train_df[column].rolling(window=12).mean()
  deGasoline = train_df[column].rolling(window=12).std()

  original = plt.plot(train_df[column], color="blue", label="Original")
  media = plt.plot(mediaGasoline, color='red', label = 'Media ' + dfToUse)
  ds = plt.plot(deGasoline, color='black', label = 'Desviación Estándar ' + dfToUse)
  plt.legend(loc = 'best')
  plt.title('Media y desviación estándar ' + column)
  plt.show(block=False)

In [ ]:
make_timeline(label_regular)

In [ ]:
train_regular = train_df[['Fecha', label_regular]]
test_regular = test_df[['Fecha', label_regular]]

# Gasolina regular
train_regular[label_regular] = train_regular[label_regular].astype(float)
train_regular_indexed = train_regular.set_index(['Fecha'])

######### TEST ###############
# Gasolina regular
test_regular[label_regular] = test_regular[label_regular].astype(float)
test_regular_indexed = test_regular.set_index(['Fecha'])

In [ ]:
# 2000-01-01 a 2019-12-01
removeCovid = 240
train_regular_indexed = train_regular_indexed[0:removeCovid]
train_regular_gas = train_regular_indexed[label_regular]

In [ ]:
train_regular_log = np.log(train_regular_gas)
plt.plot(train_regular_log)

## Test de Dickey Fuller 

In [ ]:
train_regular_gas_log_diff = train_regular_gas.diff()
train_regular_gas_log_diff.dropna(inplace=True)

In [ ]:
plt.plot(train_regular_gas_log_diff)

## Mejor modelo ARIMA 221

In [ ]:
seasonalOrder = 3
if (dfToUse == "importacion"):
  seasonalOrder = 0

model_regular_121 = SARIMAX(
  train_regular_log,
  order=(1,2,1),
  seasonal_order=(seasonalOrder,1,0,12),
  enforce_stationarity=False,
  enforce_invertibility=False
)
resultado_regular_121 = model_regular_121.fit()
print(resultado_regular_121.summary().tables[1])

In [ ]:
resultado_regular_121.plot_diagnostics(figsize=(18, 8))
plt.show()

## Prediccion

In [ ]:
df_regular_indexed = df[['Fecha', label_regular]]
df_regular_indexed =  df_regular_indexed.set_index(['Fecha'])

In [ ]:
def checkModel(prediction, test_indexed, label):
  pred = prediction.get_prediction(
    start=test_indexed.index[0],
    end=test_indexed.index[-1],
  ).summary_frame(alpha=0.05)

  fig, ax = plt.subplots(figsize=(15,5))
  test_log = np.log(test_indexed[label])
  ax = test_log.plot(label='Test Data')
  ax.set(
      title='True and Predicted Values, with Confidence Intervals',
      xlabel='Date',
      ylabel='Actual / Predicted Values'
  )

  pred['mean'].plot(ax=ax, style='r', label='Predicted Mean')
  ax.fill_between(
      pred.index, pred['mean_ci_lower'], pred['mean_ci_upper'],
      color='r', alpha=0.1
  )
  legend = ax.legend(loc='upper left')
  plt.show()
  return pred

In [ ]:
arimaPred = checkModel(resultado_regular_121, test_regular_indexed, label_regular)

# LSTM


In [ ]:
tf.random.set_seed(123)

### Test de Dickey Fuller

In [ ]:
indexed_df = df.set_index(['Fecha'])

In [ ]:
print('Resultados del Test de Dickey Fuller')
dfTest = adfuller(indexed_df, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

In [ ]:
regular_diff = indexed_df.diff()
regular_diff.fillna(0,inplace=True)
dfTest = adfuller(regular_diff)
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

In [ ]:
plt.plot(regular_diff)

### Normalizando la serie

In [ ]:
scaler = StandardScaler()
regular_diff_scaled = scaler.fit_transform(regular_diff) 
regular_diff_scaled[1:7]

### Separando train y test

In [ ]:
rows = len(regular_diff_scaled)
train = regular_diff_scaled[0:rows-17] # 2000-01-01 a 2020-12-01
test = regular_diff_scaled[rows-17:] # 2021-01-01	 a 2022-05-01	
train=np.insert(train,0,0)
train=np.reshape(train,(train.shape[0],1))
print(len(train), len(test))


### Serie supervisada

In [ ]:
# Corriendo los valores una posicion
from operator import concat


def supervisada(serie,retrasos = 1):
    serie_x = []
    serie_y = []
    for i in range(len(serie)-retrasos):
        valor = serie[i:(i+retrasos),0]
        valor_sig = serie[i+retrasos,0]
        serie_x.append(valor)
        serie_y.append(valor_sig)
    return np.array(serie_x), np.array(serie_y)

x_train,y_train = supervisada(train)
x_test,y_test = supervisada(test)




### Creando el modelo
Para que pueda hacerse el modelo se le tiene que suministrar una matriz de 3 dimensiones siendo estas:
- ***Muestras:*** número de observaciones en cada lote, también conocido como tamaño del lote.
- ***Pasos de tiempo:*** Pasos de tiempo separados para una observación dada. En este ejemplo los pasos de tiempo = 1
- ***Características:*** Para un caso univariante, como en este ejemplo, las características = 1  

In [ ]:
x_train = np.reshape(x_train,(x_train.shape[0],1,1))
x_test = np.reshape(x_test, (x_test.shape[0],1,1))

In [ ]:
def createModel(lote, paso, caracteristicas=1, dense=1):
    modelo = Sequential()
    modelo.add(LSTM(lote, batch_input_shape=(lote,paso,caracteristicas),stateful=True))
    modelo.add(Dense(dense))
    modelo.summary()
    
    modelo.compile(loss='mean_squared_error',optimizer="rmsprop")

    return modelo


def graphLoss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='center')

def showLoss(modelo):
    print("Pérdida en Entrenamiento")
    modelo.evaluate(
        x = x_train,
        y = y_train
    )
    print("Pérdida en Validación")
    modelo.evaluate(
        x = x_val,
        y = y_val
    )
    print("Pérdida en Prueba")
    modelo.evaluate(
        x = x_test,
        y = y_test
    )

In [ ]:
def modelToUse(index):
    if index == 1:
        return [1, 1, 50, 1]
    else:
        return [1, 1, 100, 4]

lote, paso, epocas, dense = modelToUse(1)
model = createModel(lote, paso, dense=dense)

lote2, paso2, epocas2, dense2 = modelToUse(2)
model2 = createModel(lote2, paso2, dense=dense2)

### Entrenando el modelo

In [ ]:
history= model.fit(
    x = x_train,
    y = y_train,
    batch_size = lote,
    epochs = epocas,
    shuffle = False,
    verbose=1
)

history2= model2.fit(
    x = x_train,
    y = y_train,
    batch_size = lote2,
    epochs = epocas2,
    shuffle = False,
    verbose=1
)

El modelo 1 parece estancarse en la epoca 40

El modelo 2 parece estancarse en la epoca 88

### Prediccion

In [ ]:
def prediccion_fun(data,modelo, batch_size,scaler,dif=False,dif_cant=1, Series=None, n=1):
    prediccion = [0]* (len(data))
    i=0
    for X in data:
        X = np.reshape(X,(1,1,1))
        yhat = modelo.predict(X, batch_size=batch_size,verbose=0)
        # invert scaling
        yhat = scaler.inverse_transform(yhat)
        if dif:
             # invert differencing
            yhat  = yhat + Series[(n+dif_cant*i)]
        # store
        prediccion[i]=yhat[0][0]
        i = i+1
    return prediccion

In [ ]:
prediccion_val = []

prediccion_test = prediccion_fun(x_test,model, 1,scaler,dif=True,dif_cant=1, Series = indexed_df.values , n=252)
prediccion_test_2 = prediccion_fun(x_test,model2, 1,scaler,dif=True,dif_cant=1, Series = indexed_df.values , n=252)

In [ ]:
test

In [ ]:
len(prediccion_test)

In [ ]:
df_test = pd.DataFrame(prediccion_test,index=test_regular_indexed[1:].index)
df_test_2 = pd.DataFrame(prediccion_test_2,index=test_regular_indexed[1:].index)


plt.plot(indexed_df)
plt.plot(df_test)
plt.plot(df_test_2)



# Comparando modelos

In [ ]:
test_log = np.log(test_regular_indexed[label_regular])

arima_rmse_error = rmse(test_log, arimaPred['mean'])
arima_mse_error = arima_rmse_error**2
mean_value = train_regular_log.mean()

print(f'MSE Error: {arima_mse_error}\nRMSE Error: {arima_rmse_error}\nMean: {mean_value}')

In [ ]:
test_to_compare_log = np.log(df_test.astype(float))
lstm_rmse_error = rmse(test_to_compare_log[0], test_log[1:])

lstm_mse_error = lstm_rmse_error**2
mean_value = train_regular_log.mean()

print(f'MSE Error: {lstm_mse_error}\nRMSE Error: {lstm_rmse_error}\nMean: {mean_value}')

In [ ]:
test_to_compare_log_2 = np.log(df_test_2.astype(float))
lstm_rmse_error_2 = rmse(test_to_compare_log_2[0], test_log[1:])

lstm_mse_error_2 = lstm_rmse_error_2**2
mean_value = train_regular_log.mean()

print(f'MSE Error: {lstm_mse_error}\nRMSE Error: {lstm_mse_error_2}\nMean: {mean_value}')

In [ ]:
rmse_errors = [arima_rmse_error, lstm_rmse_error, lstm_mse_error_2]
mse_errors = [arima_mse_error, lstm_mse_error, lstm_mse_error_2]
errors = pd.DataFrame({"Models" : ["ARIMA", "LSTM", "LSTM 2"],"RMSE Errors" : rmse_errors, "MSE Errors" : mse_errors})
plt.figure(figsize=(16,9))
plt.plot_date(test_log.index, test_log, linestyle="-")
plt.plot_date(arimaPred.index, arimaPred['mean'], linestyle="-.")
plt.plot_date(test_to_compare_log.index, test_to_compare_log[0], linestyle="--")
plt.plot_date(test_to_compare_log_2.index, test_to_compare_log_2[0], linestyle="---")
plt.legend()
plt.show()

In [ ]:
print(f"Mean: {train_regular_log.mean()}")
errors